# Capstone

### This notebook will be used for the coursera capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<b>purpose of this section is to retrieve the boroughs and the neighbourhoods in Toronto</b>
<ul><li> step 1 : scrape wikipediapage to get a list of all neighbourhoods
<li> step 2 : extract the correct table using BeautifulSoup / clease irrelevant data until only the rows containing neighbourhoods is left
<li> step 3 : assign the neighbourhoods & associated boroughs from the wikipedia page to a dataframe 
<li> step 4 : use geocoders to find the latitude and longitude of each neighbourhood
    <br> ! only those neighourhoods that return a geolocation will be kept, the rest is discarded
</ul>

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_page = requests.get(url).text

In [4]:
soup = BeautifulSoup(raw_page, 'html.parser')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df.shape

(103, 3)

# Answer 1

In [8]:
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

Adding latitude and longitude --> extract from CSV based on postal code 
Initially used geolocator, but this sometimes returns Null for certain Postal Codes

In [9]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df2.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Test how to extract the Latitude data from df2 for a given Postal Code

In [10]:
df2.loc[df2['Postal Code']=='M1B'].Latitude.item()

43.8066863

In [11]:
Toronto_Neighbourhoods = pd.DataFrame(columns=["PostalCode","Borough","Neighbourhood","Latitude","Longitude"])
    
for PostalCode,Borough,Neighborhood in zip(df['PostalCode'],df['Borough'],df['Neighborhood']):
        latitude = df2.loc[df2['Postal Code']==PostalCode].Latitude.item()
        longitude = df2.loc[df2['Postal Code']==PostalCode].Longitude.item()
        Toronto_Neighbourhoods = Toronto_Neighbourhoods.append({"PostalCode":PostalCode,"Borough":Borough,"Neighbourhood":Neighborhood, "Latitude":latitude,"Longitude":longitude}, ignore_index=True)


In [12]:
Toronto_Neighbourhoods.shape

(103, 5)

# Answer 2

In [13]:
Toronto_Neighbourhoods.head()

PostalCode           Borough                     Neighbourhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 43.6534817, -79.3839347.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Toronto_Neighbourhoods['Latitude'], Toronto_Neighbourhoods['Longitude'], Toronto_Neighbourhoods['Borough'], Toronto_Neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_Neighbourhoods['Borough'].unique()),
        Toronto_Neighbourhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


rename dataframe to toronto_data cfr the initial lab manhattan_data for easier reuse of code

In [18]:
toronto_data = Toronto_Neighbourhoods
toronto_data.head()

PostalCode           Borough                     Neighbourhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

#toronto_data = toronto_data.drop(toronto_data.loc[toronto_data['Neighbourhood']=='Toronto Dominion Centre, Design Exchange'].index)

--> had some issues with Foursquare API on this row, but I suspect it is due to overusing the API

In [19]:
toronto_data.shape

(103, 5)

### In this section we will start to use the Foursquare API to collect the data to be able to segment the available neighbourhoos in the toronto_data dataframe

In [20]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '' # your FourSquare Access Token
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: NRGWPWIMA5AWCAV21UY2J0FZWB2G5ER13JOFYBLBUXE2YTEK
CLIENT_SECRET:KA31EXTMLTER5UTDCV3G1ZYOHGWOK5WRJKPS1NHNK4I5D1H0


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
nearby_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [23]:
toronto_venues=nearby_venues
print(toronto_venues.shape)
toronto_venues.head()

(2106, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop

In [24]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                             
Agincourt                                                            4   
Alderwood, Long Branch                                               7   
Bathurst Manor, Wilson Heights, Downsview North                     23   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   25   
...                                                                ...   
Willowdale South                                                    35   
Willowdale West                                                      6   
Woburn                                                               3   
Woodbine Heights                                                     7   
York Mills West                                                      3   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                                             4   
Alderwood, Long Branch                                                7   
Bathurst Manor, Wilson Heights, Downsview North                      23   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    25   
...                                                                 ...   
Willowdale South                                                     35   
Willowdale West                                                       6   
Woburn                                                                3   
Woodbine Heights                                                      7   
York Mills West                                                       3   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Agincourt                                            4               4   
Alderwood, Long Branch                               7               7   
Bathurst Manor, Wilson Heights, Downsview North     23              23   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   25              25   
...                                                ...             ...   
Willowdale South                                    35              35   
Willowdale West                                      6               6   
Woburn                                               3               3   
Woodbine Heights                                     7               7   
York Mills West                                      3               3   

                                                 Venue Longitude  \
Neighborhood                                                       
Agincourt                                                      4   
Alderwood, Long Branch                                         7   
Bathurst Manor, Wilson Heights, Downsview North               23   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             25   
...                                                          ...   
Willowdale South                                              35   
Willowdale West                                                6   
Woburn                                                         3   
Woodbine Heights                                               7   
York Mills West                                                3   

                                                 Venue Category  
Neighborhood                                                     
Agincourt                                                     4  
Alderwood, Long Branch                                        7 

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighbourhood  Accessories Store  Airport  Airport Food Court  \
0         Parkwoods                  0        0                   0   
1         Parkwoods                  0        0                   0   
2  Victoria Village                  0        0                   0   
3  Victoria Village                  0        0                   0   
4  Victoria Village                  0        0                   0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  ...  Turkish Restaurant  \
0             0         0  ...                   0   
1             0         0  ...                   0   
2             0         0  ...                   0   
3             0         0  ...                   0   
4             0         0  ...                   0   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
0                      0                0         0            0   
1                      0                0         0            0   
2                      0                0         0            0   
3                      0                0         0            0   
4                      0                0         0            0   

   Women's Store  Yoga Studio  
0              0            0  
1              0            0  
2              0            0  
3              0            0  
4              0            0  

[5 rows x 271 columns]

In [27]:
toronto_onehot.shape

(2106, 271)

In [67]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  Accessories Store  \
0                                         Agincourt                0.0   
1                            Alderwood, Long Branch                0.0   
2   Bathurst Manor, Wilson Heights, Downsview North                0.0   
3                                   Bayview Village                0.0   
4                 Bedford Park, Lawrence Manor East                0.0   
..                                              ...                ...   
93                                 Willowdale South                0.0   
94                                  Willowdale West                0.0   
95                                           Woburn                0.0   
96                                 Woodbine Heights                0.0   
97                                  York Mills West                0.0   

    Airport  Airport Food Court  Airport Lounge  Airport Service  \
0       0.0                 0.0             0.0              0.0   
1       0.0                 0.0             0.0              0.0   
2       0.0                 0.0             0.0              0.0   
3       0.0                 0.0             0.0              0.0   
4       0.0                 0.0             0.0              0.0   
..      ...                 ...             ...              ...   
93      0.0                 0.0             0.0              0.0   
94      0.0                 0.0             0.0              0.0   
95      0.0                 0.0             0.0              0.0   
96      0.0                 0.0             0.0              0.0   
97      0.0                 0.0             0.0              0.0   

    Airport Terminal  American Restaurant  Antique Shop  Aquarium  ...  \
0                0.0                 0.00           0.0       0.0  ...   
1                0.0                 0.00           0.0       0.0  ...   
2                0.0                 0.00           0.0       0.0  ...   
3                0.0                 0.00           0.0       0.0  ...   
4                0.0                 0.04           0.0       0.0  ...   
..               ...                  ...           ...       ...  ...   
93               0.0                 0.00           0.0       0.0  ...   
94               0.0                 0.00           0.0       0.0  ...   
95               0.0                 0.00           0.0       0.0  ...   
96               0.0                 0.00           0.0       0.0  ...   
97               0.0                 0.00           0.0       0.0  ...   

    Turkish Restaurant  Vegetarian / Vegan Restaurant  Video Game Store  \
0                  0.0                            0.0               0.0   
1                  0.0                            0.0               0.0   
2                  0.0                            0.0               0.0   
3                  0.0                            0.0               0.0   
4                  0.0                            0.0               0.0   
..                 ...                            ...               ...   
93                 0.0                            0.0               0.0   
94                 0.0                            0.0               0.0   
95                 0.0                            0.0               0.0   
96                 0.0                            0.0               0.0   
97                 0.0                            0.0               0.0   

    Video Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0      0.000000               0.000000              0.0       0.0   
1      0.000000               0.000000              0.0       0.0   
2      0.000000               0.000000              0.0       0.0   
3      0.000000               0.000000              0.0       0.0   
4      0.000000               0.000000              0.0       0.0   
..          ...                    ...              ...       ...   
93     0.000000               0.028571              0.0    

In [68]:
toronto_grouped.shape

(98, 271)

In [69]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

;

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3     Coffee Shop  0.14
4             Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2      Gift Shop  0.04
3  Shopping Mall  0.04
4    Bridal Shop  0.04


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4           Nail Salon  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3         Pizza Place  0.04
4    Greek Rest

4  Mediterranean Restaurant   0.2


----Humber Summit----
                             venue  freq
0                      Pizza Place  0.33
1                     Intersection  0.33
2                              Gym  0.33
3  Molecular Gastronomy Restaurant  0.00
4                           Museum  0.00


----Humberlea, Emery----
               venue  freq
0     Baseball Field   0.5
1       Food Service   0.5
2  Accessories Store   0.0
3  Mobile Phone Shop   0.0
4              Motel   0.0


----Humewood-Cedarvale----
               venue  freq
0       Hockey Arena  0.25
1              Field  0.25
2             Garden  0.25
3              Trail  0.25
4  Accessories Store  0.00


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Pizza Place  0.06
2      Sushi Restaurant  0.06
3        Sandwich Place  0.06
4          Liquor Store  0.06


----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0          Ho

               venue  freq
0               Park   1.0
1  Accessories Store   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern Restaurant  0.29
1                     Bakery  0.14
2             Sandwich Place  0.14
3                 Smoke Shop  0.14
4      Vietnamese Restaurant  0.14


----Willowdale South----
              venue  freq
0       Coffee Shop  0.09
1  Ramen Restaurant  0.09
2       Pizza Place  0.06
3              Café  0.06
4  Sushi Restaurant  0.06


----Willowdale West----
           venue  freq
0    Pizza Place  0.17
1       Pharmacy  0.17
2        Butcher  0.17
3  Grocery Store  0.17
4    Coffee Shop  0.17


----Woburn----
                   venue  freq
0            Coffee Shop  0.67
1  Korean BBQ Restaurant  0.33
2      Accessories Store  0.00
3    Monument / Landmark  0.00
4                 Museum  0.00


----Woodbine Heights----
          venue  freq
0

''

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                                        Agincourt        Breakfast Spot   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Bank   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

  2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0                Lounge  Latin American Restaurant          Skating Rink   
1                   Gym             Sandwich Place           Coffee Shop   
2                  Bank                  Gift Shop         Shopping Mall   
3    Chinese Restaurant        Japanese Restaurant                  Café   
4        Sandwich Place         Italian Restaurant           Pizza Place   

  5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
0    Mexican Restaurant  Middle Eastern Restaurant         Metro Station   
1                   Pub                   Pharmacy  Other Great Outdoors   
2           Bridal Shop          Mobile Phone Shop   Fried Chicken Joint   
3            Nail Salon                Music Venue                Museum   
4      Greek Restaurant           Sushi Restaurant            Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0    Miscellaneous Shop                Museum      Mobile Phone Shop  
1             Pet Store           Men's Store          Metro Station  
2    Frozen Yogurt Shop        Sandwich Place            Supermarket  
3         Movie Theater                 Motel      Accessories Store  
4             Juice Bar       Thai Restaurant                    Pub

In [72]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 4, 1, 4, 1, 1, 0, 4, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1,
       4, 1, 1, 4, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 3])

In [73]:
len(kmeans.labels_)

98

In [74]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

PostalCode           Borough                     Neighbourhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  Cluster Labels 1st Most Common Venue   2nd Most Common Venue  \
0 -79.329656             4.0     Food & Drink Shop                    Park   
1 -79.315572             1.0           Pizza Place            Hockey Arena   
2 -79.360636             1.0           Coffee Shop                  Bakery   
3 -79.464763             1.0        Clothing Store  Furniture / Home Store   
4 -79.389494             1.0           Coffee Shop        Sushi Restaurant   

   3rd Most Common Venue 4th Most Common Venue  \
0      Accessories Store     Mobile Phone Shop   
1  Portuguese Restaurant           Coffee Shop   
2                   Park                  Café   
3      Accessories Store              Boutique   
4            Yoga Studio               Theater   

             5th Most Common Venue 6th Most Common Venue  \
0                            Motel   Moroccan Restaurant   
1  Molecular Gastronomy Restaurant                Museum   
2                   Breakfast Spot                   Pub   
3            Vietnamese Restaurant    Miscellaneous Shop   
4                    Smoothie Shop         Burrito Place   

  7th Most Common Venue            8th Most Common Venue  \
0   Monument / Landmark  Molecular Gastronomy Restaurant   
1         Movie Theater                            Motel   
2            Restaurant                          Theater   
3           Coffee Shop              Monument / Landmark   
4                  Café               Mexican Restaurant   

        9th Most Common Venue     10th Most Common Venue  
0  Modern European Restaurant  Middle Eastern Restaurant  
1         Moroccan Restaurant        Monument / Landmark  
2                 Yoga Studio             Chocolate Shop  
3               Movie Theater                      Motel  
4         Fried Chicken Joint             Sandwich Place

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster 1

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
6   Scarborough             0.0  Fast Food Restaurant     Accessories Store   
27   North York             0.0               Dog Run                  Pool   

   3rd Most Common Venue 4th Most Common Venue     5th Most Common Venue  \
6            Music Venue           Men's Store             Metro Station   
27           Golf Course  Fast Food Restaurant  Mediterranean Restaurant   

   6th Most Common Venue      7th Most Common Venue 8th Most Common Venue  \
6     Mexican Restaurant  Middle Eastern Restaurant    Miscellaneous Shop   
27    Mexican Restaurant  Middle Eastern Restaurant    Miscellaneous Shop   

   9th Most Common Venue      10th Most Common Venue  
6      Mobile Phone Shop  Modern European Restaurant  
27     Mobile Phone Shop                 Music Venue

### cluster 2

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
1               North York             1.0                 Pizza Place   
2         Downtown Toronto             1.0                 Coffee Shop   
3               North York             1.0              Clothing Store   
4             Queen's Park             1.0                 Coffee Shop   
7               North York             1.0  Construction & Landscaping   
..                     ...             ...                         ...   
97        Downtown Toronto             1.0                 Coffee Shop   
99        Downtown Toronto             1.0            Sushi Restaurant   
100  East Toronto Business             1.0          Light Rail Station   
101              Etobicoke             1.0              Baseball Field   
102              Etobicoke             1.0           Convenience Store   

      2nd Most Common Venue  3rd Most Common Venue  \
1              Hockey Arena  Portuguese Restaurant   
2                    Bakery                   Park   
3    Furniture / Home Store      Accessories Store   
4          Sushi Restaurant            Yoga Studio   
7       Japanese Restaurant   Caribbean Restaurant   
..                      ...                    ...   
97                     Café                  Hotel   
99              Coffee Shop    Japanese Restaurant   
100             Yoga Studio          Garden Center   
101               Locksmith      Accessories Store   
102                     Gym   Fast Food Restaurant   

          4th Most Common Venue            5th Most Common Venue  \
1                   Coffee Shop  Molecular Gastronomy Restaurant   
2                          Café                   Breakfast Spot   
3                      Boutique            Vietnamese Restaurant   
4                       Theater                    Smoothie Shop   
7                          Café                              Gym   
..                          ...                              ...   
97          Japanese Restaurant                              Gym   
99                      Gay Bar                       Restaurant   
100                        Park                       Comic Shop   
101  Modern European Restaurant                    Movie Theater   
102              Sandwich Place                  Supplement Shop   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
1                  Museum         Movie Theater                 Motel   
2                     Pub            Restaurant               Theater   
3      Miscellaneous Shop           Coffee Shop   Monument / Landmark   
4           Burrito Place                  Café    Mexican Restaurant   
7     Moroccan Restaurant                Museum         Movie Theater   
..                    ...                   ...                   ...   
97             Restaurant           Salad Place   American Restaurant   
99            Yoga Studio                   Pub           Men's Store   
100      Recording Studio            Restaurant        Farmers Market   
101                 Motel   Moroccan Restaurant   Monument / Landmark   
102            Kids Store         Tanning Salon                Bakery   

               9th Most Common Venue    10th Most Common Venue  
1                Moroccan Restaurant       Monument / Landmark  
2                        Yoga Studio            Chocolate Shop  
3                      Movie Theater                     Motel  
4                Fried Chicken Joint            Sandwich Place  
7                              Motel       Monument / Landmark  
..                               ...                       ...  
97                Seafood Restaurant                Steakhouse  
99                     Grocery Store  Mediterranean Restaurant  
100             Fast Food Restaurant                Skate Park  
101  Molecular Gastronomy Restaurant         Mobile Phone Shop  
102                   Discount Store             Grocery Store  

[79 ro

### cluster 3

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
62  Central Toronto             2.0                Garden   

   2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
62     Accessories Store  Modern European Restaurant         Movie Theater   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
62                 Motel   Moroccan Restaurant   Monument / Landmark   

              8th Most Common Venue 9th Most Common Venue  \
62  Molecular Gastronomy Restaurant     Mobile Phone Shop   

   10th Most Common Venue  
62         Massage Studio

### cluster 4

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
21             York             3.0                  Park   
64             York             3.0                  Park   
66       North York             3.0                  Park   
83  Central Toronto             3.0                  Park   

   2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
21         Women's Store                        Pool     Accessories Store   
64     Accessories Store           Mobile Phone Shop         Movie Theater   
66     Convenience Store  Modern European Restaurant                Museum   
83     Accessories Store           Mobile Phone Shop         Movie Theater   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
21     Mobile Phone Shop                 Motel   Moroccan Restaurant   
64                 Motel   Moroccan Restaurant   Monument / Landmark   
66         Movie Theater                 Motel   Moroccan Restaurant   
83                 Motel   Moroccan Restaurant   Monument / Landmark   

              8th Most Common Venue            9th Most Common Venue  \
21              Monument / Landmark  Molecular Gastronomy Restaurant   
64  Molecular Gastronomy Restaurant       Modern European Restaurant   
66              Monument / Landmark  Molecular Gastronomy Restaurant   
83  Molecular Gastronomy Restaurant       Modern European Restaurant   

        10th Most Common Venue  
21  Modern European Restaurant  
64          Miscellaneous Shop  
66           Mobile Phone Shop  
83          Miscellaneous Shop

### cluster 5

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
0               North York             4.0           Food & Drink Shop   
10              North York             4.0                      Bakery   
32             Scarborough             4.0                  Playground   
35  East York/East Toronto             4.0           Convenience Store   
40              North York             4.0  Construction & Landscaping   
46              North York             4.0               Shopping Mall   
49              North York             4.0  Construction & Landscaping   
61         Central Toronto             4.0                    Bus Line   
77               Etobicoke             4.0              Sandwich Place   
85             Scarborough             4.0                        Park   
91        Downtown Toronto             4.0                        Park   
98               Etobicoke             4.0                        Park   

   2nd Most Common Venue       3rd Most Common Venue  \
0                   Park           Accessories Store   
10           Pizza Place         Japanese Restaurant   
32         Jewelry Store  Modern European Restaurant   
35                  Park  Modern European Restaurant   
40                  Park           Other Repair Shop   
46                  Park               Grocery Store   
49      Basketball Court                        Park   
61                  Park                 Swim School   
77     Mobile Phone Shop                        Park   
85            Playground                Intersection   
91            Playground                       Trail   
98                 River                  Smoke Shop   

         4th Most Common Venue            5th Most Common Venue  \
0            Mobile Phone Shop                            Motel   
10                        Park  Molecular Gastronomy Restaurant   
32               Movie Theater                            Motel   
35                      Museum                    Movie Theater   
40                     Airport                         Wine Bar   
46                        Bank                Accessories Store   
49                      Bakery              Monument / Landmark   
61            Business Service       Modern European Restaurant   
77               Movie Theater                            Motel   
85  Modern European Restaurant                    Movie Theater   
91  Modern European Restaurant                    Movie Theater   
98           Accessories Store                Mobile Phone Shop   

   6th Most Common Venue 7th Most Common Venue  \
0    Moroccan Restaurant   Monument / Landmark   
10                Museum         Movie Theater   
32   Moroccan Restaurant   Monument / Landmark   
35                 Motel   Moroccan Restaurant   
40                Museum        Medical Center   
46         Movie Theater                 Motel   
49           Music Venue                Museum   
61                 Motel   Moroccan Restaurant   
77   Moroccan Restaurant   Monument / Landmark   
85                 Motel   Moroccan Restaurant   
91                 Motel   Moroccan Restaurant   
98                 Motel   Moroccan Restaurant   

              8th Most Common Venue            9th Most Common Venue  \
0   Molecular Gastronomy Restaurant       Modern European Restaurant   
10                            Motel              Moroccan Restaurant   
32  Molecular Gastronomy Restaurant                Mobile Phone Shop   
35              Monument / Landmark  Molecular Gastronomy Restaurant   
40         Mediterranean Restaurant                      Men's Store   
46              Moroccan Restaurant              Monument / Landmark   
49                    Movie Theater                            Motel   
61              Monument / Landmark  Molecular Gastronomy Restaurant   
77  Molecular Gastronomy Restaurant       Modern European Restaurant   
85              Monument / Landmark  Molecular Gastronomy Restaurant   
91          